In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
cd /content/drive/MyDrive/ENDProgram/Session14

/content/drive/MyDrive/ENDProgram/Session14


In [3]:
!python -m spacy download en

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.7/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.7/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')


In [4]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from __future__ import unicode_literals

import csv
import random
import re
import os
import math
import time
import unicodedata
import codecs
from io import open
import itertools


import torch
import torch.nn as nn
import torch.optim as optim
from torch.jit import script, trace

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

import spacy
import numpy as np
import pandas as pd

In [5]:
import torchtext
from torchtext.legacy.datasets import Multi30k
from torchtext.legacy.data import Field, BucketIterator,TabularDataset

In [6]:
spacy_en = spacy.load('en')
spacy_de = spacy.load('/content/drive/MyDrive/ENDProgram/Session14/Embedding_Model/word2vec_py.model')

In [6]:
print(type(spacy_en))

<class 'spacy.lang.en.English'>


In [7]:
SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [8]:
# corpus_name = "cornell movie-dialogs corpus"
# corpus = os.path.join("/content/drive/MyDrive/Dataset", corpus_name)

# def printLines(file, n=10):
#     with open(file, 'rb') as datafile:
#         lines = datafile.readlines()
#     for line in lines[:n]:
#         print(line)

# printLines(os.path.join(corpus, "movie_lines.txt"))

In [9]:
# # Splits each line of the file into a dictionary of fields
# def loadLines(fileName, fields):
#     lines = {}
#     with open(fileName, 'r', encoding='iso-8859-1') as f:
#         for line in f:
#             values = line.split(" +++$+++ ")
#             # Extract fields
#             lineObj = {}
#             for i, field in enumerate(fields):
#                 lineObj[field] = values[i]
#             lines[lineObj['lineID']] = lineObj
#     return lines


# # Groups fields of lines from `loadLines` into conversations based on *movie_conversations.txt*
# def loadConversations(fileName, lines, fields):
#     conversations = []
#     with open(fileName, 'r', encoding='iso-8859-1') as f:
#         for line in f:
#             values = line.split(" +++$+++ ")
#             # Extract fields
#             convObj = {}
#             for i, field in enumerate(fields):
#                 convObj[field] = values[i]
#             # Convert string to list (convObj["utteranceIDs"] == "['L598485', 'L598486', ...]")
#             utterance_id_pattern = re.compile('L[0-9]+')
#             lineIds = utterance_id_pattern.findall(convObj["utteranceIDs"])
#             # Reassemble lines
#             convObj["lines"] = []
#             for lineId in lineIds:
#                 convObj["lines"].append(lines[lineId])
#             conversations.append(convObj)
#     return conversations


# # Extracts pairs of sentences from conversations
# def extractSentencePairs(conversations):
#     qa_pairs = []
#     for conversation in conversations:
#         # Iterate over all the lines of the conversation
#         for i in range(len(conversation["lines"]) - 1):  # We ignore the last line (no answer for it)
#             inputLine = conversation["lines"][i]["text"].strip()
#             targetLine = conversation["lines"][i+1]["text"].strip()
#             # Filter wrong samples (if one of the lists is empty)
#             if inputLine and targetLine:
#                 qa_pairs.append([inputLine, targetLine])
#     return qa_pairs

In [10]:
# # Define path to new file
# datafile = os.path.join(corpus, "formatted_movie_lines.txt")

# delimiter = '\t'
# # Unescape the delimiter
# delimiter = str(codecs.decode(delimiter, "unicode_escape"))

# # Initialize lines dict, conversations list, and field ids
# lines = {}
# conversations = []
# MOVIE_LINES_FIELDS = ["lineID", "characterID", "movieID", "character", "text"]
# MOVIE_CONVERSATIONS_FIELDS = ["character1ID", "character2ID", "movieID", "utteranceIDs"]

# # Load lines and process conversations
# print("\nProcessing corpus...")
# lines = loadLines(os.path.join(corpus, "movie_lines.txt"), MOVIE_LINES_FIELDS)
# print("\nLoading conversations...")
# conversations = loadConversations(os.path.join(corpus, "movie_conversations.txt"),
#                                   lines, MOVIE_CONVERSATIONS_FIELDS)

# # Write new csv file
# print("\nWriting newly formatted file...")
# with open(datafile, 'w', encoding='utf-8') as outputfile:
#     writer = csv.writer(outputfile, delimiter=delimiter, lineterminator='\n')
#     for pair in extractSentencePairs(conversations):
#         writer.writerow(pair)

# # Print a sample of lines
# print("\nSample lines from file:")
# printLines(datafile)

In [11]:
# # Default word tokens
# PAD_token = 0  # Used for padding short sentences
# SOS_token = 1  # Start-of-sentence token
# EOS_token = 2  # End-of-sentence token

# class Voc:
#     def __init__(self, name):
#         self.name = name
#         self.trimmed = False
#         self.word2index = {}
#         self.word2count = {}
#         self.index2word = {PAD_token: "PAD", SOS_token: "SOS", EOS_token: "EOS"}
#         self.num_words = 3  # Count SOS, EOS, PAD

#     def addSentence(self, sentence):
#         for word in sentence.split(' '):
#             self.addWord(word)

#     def addWord(self, word):
#         if word not in self.word2index:
#             self.word2index[word] = self.num_words
#             self.word2count[word] = 1
#             self.index2word[self.num_words] = word
#             self.num_words += 1
#         else:
#             self.word2count[word] += 1

#     # Remove words below a certain count threshold
#     def trim(self, min_count):
#         if self.trimmed:
#             return
#         self.trimmed = True

#         keep_words = []

#         for k, v in self.word2count.items():
#             if v >= min_count:
#                 keep_words.append(k)

#         print('keep_words {} / {} = {:.4f}'.format(
#             len(keep_words), len(self.word2index), len(keep_words) / len(self.word2index)
#         ))

#         # Reinitialize dictionaries
#         self.word2index = {}
#         self.word2count = {}
#         self.index2word = {PAD_token: "PAD", SOS_token: "SOS", EOS_token: "EOS"}
#         self.num_words = 3 # Count default tokens

#         for word in keep_words:
#             self.addWord(word)

Trim Rare words

In [12]:
# MIN_COUNT = 3    # Minimum word count threshold for trimming

# def trimRareWords(voc, pairs, MIN_COUNT):
#     # Trim words used under the MIN_COUNT from the voc
#     voc.trim(MIN_COUNT)
#     # Filter out pairs with trimmed words
#     keep_pairs = []
#     for pair in pairs:
#         input_sentence = pair[0]
#         output_sentence = pair[1]
#         keep_input = True
#         keep_output = True
#         # Check input sentence
#         for word in input_sentence.split(' '):
#             if word not in voc.word2index:
#                 keep_input = False
#                 break
#         # Check output sentence
#         for word in output_sentence.split(' '):
#             if word not in voc.word2index:
#                 keep_output = False
#                 break

#         # Only keep pairs that do not contain trimmed word(s) in their input or output sentence
#         if keep_input and keep_output:
#             keep_pairs.append(pair)

#     print("Trimmed from {} pairs to {}, {:.4f} of total".format(len(pairs), len(keep_pairs), len(keep_pairs) / len(pairs)))
#     return keep_pairs


# # Trim voc and pairs
# pairs = trimRareWords(voc, pairs, MIN_COUNT)

In [13]:
# clean_qa =[]
# for item in pairs:
#   if len(item) == 2 :
#     clean_qa.append(item)
   
# print(len(clean_qa)/len(pairs))

### Create and save dataframe

In [14]:
# processed_data_df = pd.DataFrame(clean_qa,columns=['src','trg'])
# processed_data_df.to_csv('/content/drive/MyDrive/ENDProgram/Session14/Dataset/English_Python_DataSet.csv',index = False)

In [15]:
# processed_data_df.head()

In [8]:
def tokenize_en(text):
    """
    Tokenizes English text from a string into a list of strings
    """
    return [tok.text for tok in spacy_en.tokenizer(text)]

In [9]:
src = Field(tokenize = tokenize_en, 
            init_token = '<sos>', 
            eos_token = '<eos>', 
            lower = True, 
            batch_first = True)

trg = Field(tokenize = tokenize_en, 
            init_token = '<sos>', 
            eos_token = '<eos>', 
            lower = True, 
            batch_first = True)

fields = {'src' : ('src', src),
          'trg' : ('trg', trg)}

In [10]:
qa_data  = TabularDataset(
                            path = '/content/drive/MyDrive/ENDProgram/Session14/Dataset/Eng_Python_250.csv',
                            format = 'csv',
                            fields = fields
                            )

In [11]:
train_data, valid_data, test_data = qa_data.split([0.7,.2,.1])

In [12]:
type(train_data)

torchtext.legacy.data.dataset.Dataset

In [21]:
# print(train_data[0].shape)

In [13]:
src.build_vocab(train_data, min_freq = 2)
trg.build_vocab(train_data, min_freq = 2)

In [14]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

BATCH_SIZE = 128

train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
    (train_data, valid_data, test_data), 
     batch_size = BATCH_SIZE,
     device = device,
     sort_within_batch = True,
     sort_key = lambda x: len(x.src))

cuda


In [15]:
class Encoder(nn.Module):
    def __init__(self, 
                 input_dim, 
                 hid_dim, 
                 n_layers, 
                 n_heads, 
                 pf_dim,
                 dropout, 
                 device,
                 max_length = 250):
        super().__init__()

        self.device = device
        
        self.tok_embedding = nn.Embedding(input_dim, hid_dim)
        self.pos_embedding = nn.Embedding(max_length, hid_dim)
        
        self.layers = nn.ModuleList([EncoderLayer(hid_dim, 
                                                  n_heads, 
                                                  pf_dim,
                                                  dropout, 
                                                  device) 
                                     for _ in range(n_layers)])
        
        self.dropout = nn.Dropout(dropout)
        
        self.scale = torch.sqrt(torch.FloatTensor([hid_dim])).to(device)
        
        
    def forward(self, src, src_mask):
        
        #src = [batch size, src len]
        #src_mask = [batch size, 1, 1, src len]
        
        batch_size = src.shape[0]
        src_len = src.shape[1]
        
        pos = torch.arange(0, src_len).unsqueeze(0).repeat(batch_size, 1).to(self.device)
        
        #pos = [batch size, src len]
        
        src = self.dropout((self.tok_embedding(src) * self.scale) + self.pos_embedding(pos))
        
        #src = [batch size, src len, hid dim]
        
        for layer in self.layers:
            src = layer(src, src_mask)
            
        #src = [batch size, src len, hid dim]
            
        return src

In [16]:
class EncoderLayer(nn.Module):
    def __init__(self, 
                 hid_dim, 
                 n_heads, 
                 pf_dim,  
                 dropout, 
                 device):
        super().__init__()
        
        self.self_attn_layer_norm = nn.LayerNorm(hid_dim)
        self.ff_layer_norm = nn.LayerNorm(hid_dim)
        self.self_attention = MultiHeadAttentionLayer(hid_dim, n_heads, dropout, device)
        self.positionwise_feedforward = PositionwiseFeedforwardLayer(hid_dim, 
                                                                     pf_dim, 
                                                                     dropout)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, src, src_mask):
        
        #src = [batch size, src len, hid dim]
        #src_mask = [batch size, 1, 1, src len] 
                
        #self attention
        _src, _ = self.self_attention(src, src, src, src_mask)
        
        #dropout, residual connection and layer norm
        src = self.self_attn_layer_norm(src + self.dropout(_src))
        
        #src = [batch size, src len, hid dim]
        
        #positionwise feedforward
        _src = self.positionwise_feedforward(src)
        
        #dropout, residual and layer norm
        src = self.ff_layer_norm(src + self.dropout(_src))
        
        #src = [batch size, src len, hid dim]
        
        return src

In [17]:
class MultiHeadAttentionLayer(nn.Module):
    def __init__(self, hid_dim, n_heads, dropout, device):
        super().__init__()
        
        assert hid_dim % n_heads == 0
        
        self.hid_dim = hid_dim
        self.n_heads = n_heads
        self.head_dim = hid_dim // n_heads
        
        self.fc_q = nn.Linear(hid_dim, hid_dim)
        self.fc_k = nn.Linear(hid_dim, hid_dim)
        self.fc_v = nn.Linear(hid_dim, hid_dim)
        
        self.fc_o = nn.Linear(hid_dim, hid_dim)
        
        self.dropout = nn.Dropout(dropout)
        
        self.scale = torch.sqrt(torch.FloatTensor([self.head_dim])).to(device)
        
        
    def forward(self, query, key, value, mask = None):
        
        batch_size = query.shape[0]
        
        #query = [batch size, query len, hid dim]
        #key = [batch size, key len, hid dim]
        #value = [batch size, value len, hid dim]
                
        Q = self.fc_q(query)
        K = self.fc_k(key)
        V = self.fc_v(value)
        
        #Q = [batch size, query len, hid dim]
        #K = [batch size, key len, hid dim]
        #V = [batch size, value len, hid dim]
                
        Q = Q.view(batch_size, -1, self.n_heads, self.head_dim).permute(0, 2, 1, 3)
        K = K.view(batch_size, -1, self.n_heads, self.head_dim).permute(0, 2, 1, 3)
        V = V.view(batch_size, -1, self.n_heads, self.head_dim).permute(0, 2, 1, 3)
        
        #Q = [batch size, n heads, query len, head dim]
        #K = [batch size, n heads, key len, head dim]
        #V = [batch size, n heads, value len, head dim]
                
        energy = torch.matmul(Q, K.permute(0, 1, 3, 2)) / self.scale
        
        #energy = [batch size, n heads, query len, key len]
        
        if mask is not None:
            energy = energy.masked_fill(mask == 0, -1e10)
        
        attention = torch.softmax(energy, dim = -1)
                
        #attention = [batch size, n heads, query len, key len]
                
        x = torch.matmul(self.dropout(attention), V)
        
        #x = [batch size, n heads, query len, head dim]
        
        x = x.permute(0, 2, 1, 3).contiguous()
        
        #x = [batch size, query len, n heads, head dim]
        
        x = x.view(batch_size, -1, self.hid_dim)
        
        #x = [batch size, query len, hid dim]
        
        x = self.fc_o(x)
        
        #x = [batch size, query len, hid dim]
        
        return x, attention

In [18]:
class PositionwiseFeedforwardLayer(nn.Module):
    def __init__(self, hid_dim, pf_dim, dropout):
        super().__init__()
        
        self.fc_1 = nn.Linear(hid_dim, pf_dim)
        self.fc_2 = nn.Linear(pf_dim, hid_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x):
        
        #x = [batch size, seq len, hid dim]
        
        x = self.dropout(torch.relu(self.fc_1(x)))
        
        #x = [batch size, seq len, pf dim]
        
        x = self.fc_2(x)
        
        #x = [batch size, seq len, hid dim]
        
        return x

In [19]:
class Decoder(nn.Module):
    def __init__(self, 
                 output_dim, 
                 hid_dim, 
                 n_layers, 
                 n_heads, 
                 pf_dim, 
                 dropout, 
                 device,
                 max_length = 250):
        super().__init__()
        
        self.device = device
        
        self.tok_embedding = nn.Embedding(output_dim, hid_dim)
        self.pos_embedding = nn.Embedding(max_length, hid_dim)
        
        self.layers = nn.ModuleList([DecoderLayer(hid_dim, 
                                                  n_heads, 
                                                  pf_dim, 
                                                  dropout, 
                                                  device)
                                     for _ in range(n_layers)])
        
        self.fc_out = nn.Linear(hid_dim, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
        self.scale = torch.sqrt(torch.FloatTensor([hid_dim])).to(device)
        
        
    def forward(self, trg, enc_src, trg_mask, src_mask):
        
        #trg = [batch size, trg len]
        #enc_src = [batch size, src len, hid dim]
        #trg_mask = [batch size, 1, trg len, trg len]
        #src_mask = [batch size, 1, 1, src len]
                
        batch_size = trg.shape[0]
        trg_len = trg.shape[1]
        
        pos = torch.arange(0, trg_len).unsqueeze(0).repeat(batch_size, 1).to(self.device)
                            
        #pos = [batch size, trg len]
            
        trg = self.dropout((self.tok_embedding(trg) * self.scale) + self.pos_embedding(pos))
                
        #trg = [batch size, trg len, hid dim]
        
        for layer in self.layers:
            trg, attention = layer(trg, enc_src, trg_mask, src_mask)
        
        #trg = [batch size, trg len, hid dim]
        #attention = [batch size, n heads, trg len, src len]
        
        output = self.fc_out(trg)
        
        #output = [batch size, trg len, output dim]
            
        return output, attention

In [20]:
class DecoderLayer(nn.Module):
    def __init__(self, 
                 hid_dim, 
                 n_heads, 
                 pf_dim, 
                 dropout, 
                 device):
        super().__init__()
        
        self.self_attn_layer_norm = nn.LayerNorm(hid_dim)
        self.enc_attn_layer_norm = nn.LayerNorm(hid_dim)
        self.ff_layer_norm = nn.LayerNorm(hid_dim)
        self.self_attention = MultiHeadAttentionLayer(hid_dim, n_heads, dropout, device)
        self.encoder_attention = MultiHeadAttentionLayer(hid_dim, n_heads, dropout, device)
        self.positionwise_feedforward = PositionwiseFeedforwardLayer(hid_dim, 
                                                                     pf_dim, 
                                                                     dropout)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, trg, enc_src, trg_mask, src_mask):
        
        #trg = [batch size, trg len, hid dim]
        #enc_src = [batch size, src len, hid dim]
        #trg_mask = [batch size, 1, trg len, trg len]
        #src_mask = [batch size, 1, 1, src len]
        
        #self attention
        _trg, _ = self.self_attention(trg, trg, trg, trg_mask)
        
        #dropout, residual connection and layer norm
        trg = self.self_attn_layer_norm(trg + self.dropout(_trg))
            
        #trg = [batch size, trg len, hid dim]
            
        #encoder attention
        _trg, attention = self.encoder_attention(trg, enc_src, enc_src, src_mask)
        # query, key, value
        
        #dropout, residual connection and layer norm
        trg = self.enc_attn_layer_norm(trg + self.dropout(_trg))
                    
        #trg = [batch size, trg len, hid dim]
        
        #positionwise feedforward
        _trg = self.positionwise_feedforward(trg)
        
        #dropout, residual and layer norm
        trg = self.ff_layer_norm(trg + self.dropout(_trg))
        
        #trg = [batch size, trg len, hid dim]
        #attention = [batch size, n heads, trg len, src len]
        
        return trg, attention

In [21]:
class Seq2Seq(nn.Module):
    def __init__(self, 
                 encoder, 
                 decoder, 
                 src_pad_idx, 
                 trg_pad_idx, 
                 device):
        super().__init__()
        
        self.encoder = encoder
        self.decoder = decoder
        self.src_pad_idx = src_pad_idx
        self.trg_pad_idx = trg_pad_idx
        self.device = device
        
    def make_src_mask(self, src):
        
        #src = [batch size, src len]
        
        src_mask = (src != self.src_pad_idx).unsqueeze(1).unsqueeze(2)

        #src_mask = [batch size, 1, 1, src len]

        return src_mask
    
    def make_trg_mask(self, trg):
        
        #trg = [batch size, trg len]
        
        trg_pad_mask = (trg != self.trg_pad_idx).unsqueeze(1).unsqueeze(2)
        
        #trg_pad_mask = [batch size, 1, 1, trg len]
        
        trg_len = trg.shape[1]
        
        trg_sub_mask = torch.tril(torch.ones((trg_len, trg_len), device = self.device)).bool()
        
        #trg_sub_mask = [trg len, trg len]
            
        trg_mask = trg_pad_mask & trg_sub_mask
        
        #trg_mask = [batch size, 1, trg len, trg len]
        
        return trg_mask

    def forward(self, src, trg):
        
        #src = [batch size, src len]
        #trg = [batch size, trg len]
                
        src_mask = self.make_src_mask(src)
        trg_mask = self.make_trg_mask(trg)
        
        #src_mask = [batch size, 1, 1, src len]
        #trg_mask = [batch size, 1, trg len, trg len]
        
        enc_src = self.encoder(src, src_mask)
        
        #enc_src = [batch size, src len, hid dim]
                
        output, attention = self.decoder(trg, enc_src, trg_mask, src_mask)
        #print(trg_mask.sum())
        #output = [batch size, trg len, output dim]
        #attention = [batch size, n heads, trg len, src len]
        
        return output, attention

In [22]:
device

device(type='cuda')

In [23]:
INPUT_DIM = len(src.vocab)
OUTPUT_DIM = len(trg.vocab)
HID_DIM = 256
ENC_LAYERS = 3
DEC_LAYERS = 3
ENC_HEADS = 8
DEC_HEADS = 8
ENC_PF_DIM = 512
DEC_PF_DIM = 512
ENC_DROPOUT = 0.1
DEC_DROPOUT = 0.1

enc = Encoder(INPUT_DIM, 
              HID_DIM, 
              ENC_LAYERS, 
              ENC_HEADS, 
              ENC_PF_DIM, 
              ENC_DROPOUT, 
              device)

dec = Decoder(OUTPUT_DIM, 
              HID_DIM, 
              DEC_LAYERS, 
              DEC_HEADS, 
              DEC_PF_DIM, 
              DEC_DROPOUT, 
              device)

In [24]:
SRC_PAD_IDX = src.vocab.stoi[src.pad_token]
TRG_PAD_IDX = trg.vocab.stoi[trg.pad_token]

model = Seq2Seq(enc, dec, SRC_PAD_IDX, TRG_PAD_IDX, device).to(device)


In [25]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 5,874,045 trainable parameters


In [26]:
def initialize_weights(m):
    if hasattr(m, 'weight') and m.weight.dim() > 1:
        nn.init.xavier_uniform_(m.weight.data)

In [27]:
model.apply(initialize_weights);

In [28]:
LEARNING_RATE = 0.0005

optimizer = torch.optim.Adam(model.parameters(), lr = LEARNING_RATE)

In [29]:
criterion = nn.CrossEntropyLoss(ignore_index = TRG_PAD_IDX)

In [30]:
def train(model, iterator, optimizer, criterion, clip):
    
    model.train()
    
    epoch_loss = 0
    
    for i, batch in enumerate(iterator):
        
        src = batch.src
        trg = batch.trg
        #print(trg.shape)
        seq_len = trg.shape[1] -1 
        optimizer.zero_grad()
        
        output, _ = model(src, trg[:,:-1])
                
        #output = [batch size, trg len - 1, output dim]
        #trg = [batch size, trg len]
            
        output_dim = output.shape[-1]
            
        output = output.contiguous().view(-1, output_dim)
        trg = trg[:,1:].contiguous().view(-1)
                
        #output = [batch size * trg len - 1, output dim]
        #trg = [batch size * trg len - 1]
        #print(output.shape,trg.shape)    
        loss = criterion(output, trg)
        
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        
        epoch_loss += loss.item()/ seq_len
        
    return epoch_loss / len(iterator)

In [31]:
def evaluate(model, iterator, criterion):
    
    model.eval()
    
    epoch_loss = 0
    
    with torch.no_grad():
    
        for i, batch in enumerate(iterator):

            src = batch.src
            trg = batch.trg
            seq_len = trg.shape[1] -1

            output, _ = model(src, trg[:,:-1])
            
            #output = [batch size, trg len - 1, output dim]
            #trg = [batch size, trg len]
            
            output_dim = output.shape[-1]
            
            output = output.contiguous().view(-1, output_dim)
            trg = trg[:,1:].contiguous().view(-1)
            
            #output = [batch size * trg len - 1, output dim]
            #trg = [batch size * trg len - 1]
            
            loss = criterion(output, trg)

            epoch_loss += loss.item()/seq_len
        
    return epoch_loss / len(iterator)

In [32]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [33]:
N_EPOCHS = 10
CLIP = 1

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'model.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

Epoch: 01 | Time: 0m 5s
	Train Loss: 0.086 | Train PPL:   1.090
	 Val. Loss: 0.062 |  Val. PPL:   1.064
Epoch: 02 | Time: 0m 5s
	Train Loss: 0.064 | Train PPL:   1.066
	 Val. Loss: 0.049 |  Val. PPL:   1.050
Epoch: 03 | Time: 0m 5s
	Train Loss: 0.053 | Train PPL:   1.055
	 Val. Loss: 0.043 |  Val. PPL:   1.044
Epoch: 04 | Time: 0m 5s
	Train Loss: 0.048 | Train PPL:   1.049
	 Val. Loss: 0.039 |  Val. PPL:   1.040
Epoch: 05 | Time: 0m 4s
	Train Loss: 0.043 | Train PPL:   1.044
	 Val. Loss: 0.036 |  Val. PPL:   1.037
Epoch: 06 | Time: 0m 5s
	Train Loss: 0.040 | Train PPL:   1.041
	 Val. Loss: 0.034 |  Val. PPL:   1.035
Epoch: 07 | Time: 0m 4s
	Train Loss: 0.037 | Train PPL:   1.038
	 Val. Loss: 0.032 |  Val. PPL:   1.033
Epoch: 08 | Time: 0m 4s
	Train Loss: 0.035 | Train PPL:   1.035
	 Val. Loss: 0.031 |  Val. PPL:   1.031
Epoch: 09 | Time: 0m 5s
	Train Loss: 0.032 | Train PPL:   1.032
	 Val. Loss: 0.030 |  Val. PPL:   1.030
Epoch: 10 | Time: 0m 5s
	Train Loss: 0.030 | Train PPL:   1.030


## Chatbot using Trained Model

In [43]:
def generate_words(sentence, sentence_field, model, device,max_length=250):
  model.eval()

  tokens = [token.lower() for token in sentence]

  tokens = [sentence_field.init_token] + tokens + [sentence_field.eos_token]
  src_indexes = [sentence_field.vocab.stoi[token] for token in tokens]
  src_tensor = torch.LongTensor(src_indexes).unsqueeze(0).to(device)
  src_mask = model.make_src_mask(src_tensor)

  with torch.no_grad():
      enc_src = model.encoder(src_tensor, src_mask)

  trg_indexes = [sentence_field.vocab.stoi[sentence_field.init_token]]

  for i in range(max_length):

        trg_tensor = torch.LongTensor(trg_indexes).unsqueeze(0).to(device)

        trg_mask = model.make_trg_mask(trg_tensor)
        
        with torch.no_grad():
            output, attention = model.decoder(trg_tensor, enc_src, trg_mask, src_mask)
        
        pred_token = output.argmax(2)[:,-1].item()
        
        trg_indexes.append(pred_token)

        if pred_token == sentence_field.vocab.stoi[sentence_field.eos_token]:
            break
    
  trg_tokens = [sentence_field.vocab.itos[i] for i in trg_indexes]
  return trg_tokens[1:]

In [44]:
def chatbot(sentence_field, model, device):
    input_sentence = ''
    while(1):
        try:
            # Get input sentence
            input_sentence = input('> ')
            # Check if it is quit case
            if input_sentence == 'q' or input_sentence == 'quit': break
            # Normalize sentence
            #input_sentence = normalizeString(input_sentence)
            # Evaluate sentence
            output_words = generate_words(input_sentence, sentence_field, model, device,max_length=250)
            # Format and print response sentence
            output_words[:] = [x for x in output_words if not (x == '<eos>' or x == '<pad>' or x == '<unk>')]
            print('Bot:', ' '.join(output_words))

        except KeyError:
            print("Error: Encountered unknown word.")

In [45]:
model.load_state_dict(torch.load('model.pt'))
chatbot(src, model, device)

> write python program to add two numbers
Bot: write and the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the
> write hello world
Bot: write python python the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the th

KeyboardInterrupt: ignored